Author: Dan Shea  
Date: 2019.11.15  
Description: This function quickly scans through a blast text output and looks for alignments with gaps. It then reports those regions in a separate file.

In [1]:
import gzip
import re
import collections
import sys

In [10]:
def parse_blast_text(filename,outfile):
    with open(outfile, 'w') as ofh:
        with gzip.open(filename, mode='rt') as fh:
            key=None
            hsp=None
            for line in fh:
                line = line.strip()
                if (match:=re.match(r'^Query= (\w+)$', line)) is not None:
                    key = match.groups()[0]
                    continue
                if (match:=re.match(r'^(\w+)\s+([\w\.\+-]+)\s+([\w\.\+-]+)$', line)) is not None:
                    hsp = match.groups()[0]
                    bit = match.groups()[1]
                    evl = match.groups()[2]
                    continue
                if (match:=re.match(r'^>\s*(\w+)$', line)) is not None:
                    hsp = match.groups()[0]
                    continue
                if (match:=re.match(r'^(\w+)\s+([0-9]+)\s+([\w-]+)\s+([0-9]+)$', line)) is not None:
                    qos = match.groups()[0]
                    start = match.groups()[1]
                    aln = match.groups()[2]
                    stop = match.groups()[3]
                    if (match:=re.search(r'(-+)', line)) is not None:
                        findall = re.findall(r'(-+)', aln)
                        indels = [len(g) for g in findall]
                        indels_str = '\t'.join(map(str,indels))
                        ofh.write(f'Query={key}\tType={qos}\tStart={start}\tStop={stop}\t{indels_str}\n')
                        ofh.write(f'{aln}\n\n')
                    continue
    pass

In [11]:
a=parse_blast_text('blast.txt.gz', 'results.txt')